# 0. 라이브러리 불러오기

In [1]:
import mido
from mido import MidiFile, MidiTrack
import numpy as np
import pandas as pd
import editdistance
import os

# 1. 동일한 시간에 동시에 누르는 건반에 대해서 데이터 추출하기

In [44]:
def midi_to_dataframe(file_path):
    mid = mido.MidiFile(file_path)

    columns = ['Sec', 'Ticks', 'msg_type', 'Channel', 'Notes', 'Velocities', 'Tempo', 'diff_velocity']
    midi_df = pd.DataFrame(columns=columns)

    current_notes = []
    current_velocities = []
    current_tempo = None
    current_ticks = None
    current_time = 0
    current_msg_type = []
    prev_notes = []
    prev_velocities = []

    for msg in mid:
        if msg.type in ['note_on', 'note_off']:
            if msg.time > 0:
                # 이전 노트 및 velocity의 평균 계산
                prev_avg_velocity = sum(prev_velocities) / len(prev_velocities) if prev_velocities else 0

                # 변화 속도 계산
                if len(current_velocities) > 0:  # 노트가 있는 경우에만 계산
                    diff_velocity = sum(current_velocities) / len(current_velocities) - prev_avg_velocity
                else:
                    diff_velocity = 0 # 없으면 0으로 처리
                
                if current_msg_type:
                    midi_df = pd.concat([midi_df, pd.DataFrame([[current_time, current_ticks, current_msg_type, msg.channel, current_notes,
                                                             current_velocities, current_tempo, diff_velocity]], columns=columns)], ignore_index=True)

                prev_notes = current_notes.copy()
                prev_velocities = current_velocities.copy()

                current_notes = []
                current_velocities = []
                current_msg_type = []

            current_notes.append(msg.note)
            current_velocities.append(msg.velocity)
            current_msg_type.append(msg.type)

        elif msg.type == 'set_tempo':
            current_tempo = mido.tempo2bpm(msg.tempo)  # 템포 정보 업데이트

        current_ticks = msg.time

        current_time += msg.time

    # 마지막 노트들을 처리
    if current_notes and current_msg_type:
        prev_avg_velocity = sum(prev_velocities) / len(prev_velocities) if prev_velocities else 0
        if len(current_velocities) > 0:  # 노트가 있는 경우에만 계산
            diff_velocity = sum(current_velocities) / len(current_velocities) - prev_avg_velocity
        else:
            diff_velocity = 0
        
        midi_df = pd.concat([midi_df, pd.DataFrame([[current_time, current_ticks, current_msg_type, '', current_notes,
                                                     current_velocities, current_tempo, diff_velocity]], columns=columns)], ignore_index=True)

    return midi_df

In [45]:
result_df = midi_to_dataframe('midi_data/Fugue1.mid')
result_df.to_csv('hello1.csv', index=False)

print(result_df)

            Sec     Ticks                      msg_type Channel         Notes  \
0      0.468750  0.468750                     [note_on]       1          [60]   
1      0.937500  0.000000           [note_off, note_on]       1      [60, 62]   
2      1.406250  0.000000           [note_off, note_on]       1      [62, 64]   
3      1.875000  0.000000           [note_off, note_on]       1      [64, 65]   
4      2.578125  0.000000           [note_off, note_on]       1      [65, 67]   
..          ...       ...                           ...     ...           ...   
765   99.981427  0.000000  [note_off, note_on, note_on]       3  [74, 79, 76]   
766  102.961690  2.980262                    [note_off]       1          [48]   
767  103.001164  0.039474                    [note_off]       0          [60]   
768  103.119585  0.118421                    [note_off]       1          [84]   
769  103.139321  0.000000          [note_off, note_off]              [76, 79]   

       Velocities      Temp

In [46]:
result_df = midi_to_dataframe('midi_data/Fugue3.mid')
result_df.to_csv('hello2.csv', index=False)

print(result_df)

            Sec     Ticks                                           msg_type  \
0      1.097560  1.097560                                          [note_on]   
1      1.463414  0.000000                                [note_off, note_on]   
2      1.646341  0.000000                                [note_off, note_on]   
3      1.829268  0.000000                                [note_off, note_on]   
4      2.012194  0.000000                                [note_off, note_on]   
..          ...       ...                                                ...   
892  159.584519  0.000000                     [note_off, note_off, note_off]   
893  159.723864  0.098361               [note_on, note_on, note_on, note_on]   
894  160.094554  0.000000  [note_off, note_off, note_off, note_off, note_...   
895  160.137657  0.000000      [note_on, note_on, note_on, note_on, note_on]   
896  162.206623  0.000000  [note_off, note_off, note_off, note_off, note_...   

    Channel                 Notes      

In [47]:
result_df = midi_to_dataframe('midi_data/sample_midi.mid')
result_df.to_csv('hello3.csv', index=False)

print(result_df)

          Sec      Ticks                                       msg_type  \
0    2.748438   0.748438  [note_on, note_on, note_on, note_on, note_on]   
1    3.131250   0.382812                                     [note_off]   
2    3.139063   0.007812                                     [note_off]   
3    3.146875   0.007812                                     [note_off]   
4    3.150000   0.003125                                     [note_off]   
..        ...        ...                                            ...   
84  23.029688   0.029688  [note_on, note_on, note_on, note_on, note_on]   
85  25.315625   2.285938                                     [note_off]   
86  25.321875   0.000000                           [note_off, note_off]   
87  25.329688   0.007812                                     [note_off]   
88  59.000000  33.643750                                     [note_off]   

   Channel                 Notes            Velocities  Tempo  diff_velocity  
0        0  [41, 53,

In [48]:
result_df = midi_to_dataframe('midi_data/sample_midi.mid')
result_df.to_csv('hello4.csv', index=False)

print(result_df)

          Sec      Ticks                                       msg_type  \
0    2.748438   0.748438  [note_on, note_on, note_on, note_on, note_on]   
1    3.131250   0.382812                                     [note_off]   
2    3.139063   0.007812                                     [note_off]   
3    3.146875   0.007812                                     [note_off]   
4    3.150000   0.003125                                     [note_off]   
..        ...        ...                                            ...   
84  23.029688   0.029688  [note_on, note_on, note_on, note_on, note_on]   
85  25.315625   2.285938                                     [note_off]   
86  25.321875   0.000000                           [note_off, note_off]   
87  25.329688   0.007812                                     [note_off]   
88  59.000000  33.643750                                     [note_off]   

   Channel                 Notes            Velocities  Tempo  diff_velocity  
0        0  [41, 53,